# Partially Supervised Feature Selection with Regularized Linear Models


## Feature selection methods overview

This item is based con the first paper.

**Goals of feature selection**

Scenarios related to few tens of samples but thousands dimensions: microarray data, 

1. To avoid overfiting and improve model performance, prediction performance in the case of supervised classification and better cluster detection in unsupervised scenarios.

2. To provide more efficient models

3. To gain a deeper insight into the underlying processes that generated the data. The excess of dimensionality difficult the understanding.

The problem is related to find the optimal model parameters for the optimal feature subset. So, the model parameters becomes dependent of the features selected and need to be computed more or less coupled with the guessing of model parameters.

From less (zero) to more coupled computation, we have three strategies:

1. Filter techniques. Two step process, first the filtering, then the training of the model. Take into account only the properties of the data and in some cases a certain amount of prior knowledge. Therefore it's independent of the classification method. In its most simplest form ignores dependences on the data (univariate).

    Examples: Euclidean distance, i-test Information gain, Markov blanket filter

2. Wrapper methods. Once selected a candidate subset of features, the classification model is evaluated by training and testing the model. This is iterated over a ensemble of candidate subsets, and the model (with his feature subsets) selected is the model with the best accuracy. 
    
    It's very important to construct a good searching algorithm of subsets, in order to reduce the number of sets to model with. This methods are dependent of the classifier, model feature dependencies and have the risk to be bind to a local optima. With randomizing techniques this problem is bypassed to some extent. 
    
    Examples: Sequential forward selection (SFS) , Sequential backward elimination, Simulated annealing, Randomized hill climbing, Genetic algorithms.

3. Embedded methods. The search of the optimal subset of features is built into the classifier. Have the advantage that they include the interaction with the classification model, while at the same time being far less computationally intensive than wrapper methods.

    Examples: Decision trees Weighted naive Bayes, Feature selection using the weight vector of SVM, AROM
    
### AROM methods

The acronym derives from *Approximation of Minimization zeRO-norm*

The problem is obtain a linear predictor $h$, minimizing the number of independent variables (features) without loss of accuracy:

$$h(\mathbf{x}) = sign(\mathbf{w} \cdot \mathbf{x} + b)$$

for $n$ samples $x_i \in \mathbb{R}^n$ and $m$ labels $y_i \in \{\pm1\}$.

The accuracy constraint requires correspondence of sign 

$sign(y_i) \cdot sign(h_i) > 0$ or in other form $y_i \cdot h_i = 1$

or less restrictive, enabling $\mathbf{w}$ to scale freely $y_i \cdot h_i \ge 1$

so 

$$y_i(\mathbf{w} \cdot \mathbf{x} + b) \ge 1$$

The minimization is done with a norm defined over the vectorial space of $\mathbf{w}$. One approach is to minimize the zero-norm, that is, the number of components of the vector (number of non null $w_i$). But it's know to be a NP-Hard problem.

It's more adequate compute over a 1-norm or a 2-norm. In the second paper, the author deduce a suitable form for the function that could be minimized, taken into account the former constraint:

$$\displaystyle\sum_{j=1}^n ln(|w_j| + \epsilon)$$

The term $\epsilon$ is included to protect from zero values inside logarithm.

AROM methods are therefore feature selection embedded methods.

**l1-AROM** and **l2-AROM** (in this case by means of a 2-norm minimization) algorithms optimize this algorithm by iterative rescaling of inputs and doing a smooth feature selection since the weight coefficients along some dimensions progressively drop below the machine precision while other dimensions become more significant.

### AROM semi-supervised

Third and Fourth papers explore a improvement of these previous described methods.

**Goal**

Classification of microarray data: few tens of samples against several thousand dimensions (genes).

**Key differential strategy**

Extend AROM methods by means of partial supervision on the dimensions of a feature selection procedure. The technique proposes to use of prior knowledge to guide feature selection, but flexible enough to let the final selection depart from it if necessary to optimize the classification objective.

The preferential features are previously selected from similar datasets in large microarray databases because it's known that different sub-samples of patients lead to very similar sets of biomarkers, as expected if we are aware that the biological process explaining the outcome is common among different patients.

This datasets are called source datasets and we expect that the prediction for a similar feature vector is the same than the prediction for this vector in our dataset (the target).

*In third paper prior knowledge is incorporated by biological information*

So, if we have some knowledge on the relative importance of each feature (either from actual prior knowledge or from a related dataset), the supervised AROM objective can be modified by adding a prior relevance vector $\beta = [\beta_1,...,\beta_n]$  defined over the $n$ dimensions and where $\beta_j >0$ is the prior relevance of the $j$ feature.

So in this case, the function to minimize in the case of 1-norm is:

$$\displaystyle\sum_{j=1}^n \frac{1}{\beta_j} ln(|w_j| + \epsilon)$$


## L2-AROM
Describe how the provided implementation of L2-AROM works. See [2, 3, 4] for specific details. Next, implement a variable ranking approach based on the PS-L2-AROM method, as described in [4], using the provided implementation of L2-AROM.

You should introduce the possibility in the previous implementation to specify the initial value of of the scaling vector z. By default this vector should be equal to a vector with all components equal to one. By increasing or reducing these values, one should be able to favor, or make more difficult the selection of specific features. This will lead to the method PS-L2-AROM, in which some sort of prior-knowledge about the importance of each feature can be considered.

### Implementation

**SVM**

Suppose some given data points each belong to one of two classes, and the goal is to decide which class a new data point will be in. In the case of support-vector machines, a data point is viewed as a {\displaystyle p} p-dimensional vector (a list of {\displaystyle p} p numbers), and we want to know whether we can separate such points with a {\displaystyle (p-1)} (p-1)-dimensional hyperplane. This is called a linear classifier. There are many hyperplanes that might classify the data. One reasonable choice as the best hyperplane is the one that represents the largest separation, or margin, between the two classes. So **we choose the hyperplane so that the distance from it to the nearest data point on each side is maximized**. If such a hyperplane exists, it is known as the maximum-margin hyperplane and the linear classifier it defines is known as a maximum-margin classifier.

But often the target are not linearly separable in that space. For this reason,  the original finite-dimensional space be mapped into a much higher-dimensional space, making the separation viable in the new space. To keep the computational load reasonable, the mappings used by SVM schemes are designed to ensure that dot products of pairs of input data vectors may be computed easily in terms of the variables in the original space, by defining them in terms of a kernel function {\displaystyle k(x,y)} {\displaystyle k(x,y)} selected to suit the problem.[5] 

The hyperplanes in the higher-dimensional space are defined as the set of points whose dot product with a vector in that space is constant, where such a set of vector is an orthogonal (and thus minimal) set of vectors that defines a hyperplane. The vectors defining the hyperplanes can be chosen to be linear combinations with parameters {\displaystyle \alpha _{i}} \alpha _{i} of images of feature vectors {\displaystyle x_{i}} x_{i} that occur in the data base.[clarification needed] With this choice of a hyperplane, the points {\displaystyle x} x in the feature space that are mapped into the hyperplane are defined by the relation {\displaystyle \textstyle \sum _{i}\alpha _{i}k(x_{i},x)={\text{constant}}.} {\displaystyle \textstyle \sum _{i}\alpha _{i}k(x_{i},x)={\text{constant}}.} Note that if {\displaystyle k(x,y)} {\displaystyle k(x,y)} becomes small as {\displaystyle y} y grows further away from {\displaystyle x} x, each term in the sum measures the degree of closeness of the test point {\displaystyle x} x to the corresponding data base point {\displaystyle x_{i}} x_{i}. In this way, the sum of kernels above can be used to measure the relative nearness of each test point to the data points originating in one or the other of the sets to be discriminated. Note the fact that the set of points {\displaystyle x} x mapped into any hyperplane can be quite convoluted as a result, allowing much more complex discrimination between sets that are not convex at all in the original space.

**RFE**

Given an external estimator that assigns weights to features (e.g., the coefficients of a linear model), recursive feature elimination (RFE) is to select features by recursively considering smaller and smaller sets of features. First, the estimator is trained on the initial set of features and the importance of each feature is obtained either through a coef_ attribute or through a feature_importances_ attribute. Then, the least important features are pruned from current set of features.That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached.

RFECV performs RFE in a cross-validation loop to find the optimal number of features.


**L2-AROM**

In the present work we rely on another embedded selection method with linear models, called l1-AROM [25]. This specific choice is motivated by the possibil- ity to extend this approach in a simple yet efficient way to perform transfer learning by biasing the optimization procedure towards certain dimensions. We proposed recently such a partially supervised (PS) extension [26] but the favored dimensions were then defined from prior knowledge. In the context of microarray data, molecular biologists may indeed sometimes guess that a few genes should be considered a priori more relevant. In the present work, we do not use such prior knowledge but rather related datasets, hence performing inductive trans- fer learning at the feature level. The additional benefits are a fully automated feature selection procedure and the possibility to choose the number of features to be transferred independently of some expert knowledge. A practical approx- imation of this technique reduces to learn linear SVMs with iterative rescaling of the inputs. The rescaling factors depend here on previously selected features from existing datasets.

At step k = 0, initialize wk = β Iterate until convergence:
1 minw ||w||2
subject to: yi (w · (xi ∗ wk ) + b) ≥ 1
2 Let (w ̄ ) be the solution, set wk+1 ← wk ∗w ̄ ∗β
 
**SVC**
C-Support Vector Classification.
The implementation is based on libsvm. The fit time complexity is more than quadratic with the number of samples which makes it hard to scale to dataset with more than a couple of 10000 samples.
The multiclass support is handled according to a one-vs-one scheme.
For details on the precise mathematical formulation of the provided kernel functions and how gamma, coef0 and degree affect each other, see the corresponding section in the narrative documentation: Kernel functions.



In [1]:
import numpy as np
import pandas as pd
#from sklearn import preprocessing
from sklearn.model_selection import train_test_split
#from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
#from sklearn.feature_selection import RFE

import warnings
warnings.filterwarnings("ignore")

##
# X is numpy array witht the data (rows are data instances)
# Y is a numpy vector with the class labels (-1 or 1)
# C is the regularization coefficient of the SVM
# threshold is the threshold value to drop features in L2AROM

def variable_ranking(X, Y, C = 1, threshold = 1e-10):
    """
    """

    # Copy X to modify it later

    final_X = X.copy()

    # Initialice w_k = (1,....,1)
    z = np.ones(X.shape[1])

    # Number of attributes

    length = z.shape[0]
    print("#Features", length)

    # Array that stores the elimination order, being the higher number the first attribute 
    # that is eliminated and 1 the last one

    elimination_order = np.zeros(length, dtype = int)
    print("#Elimination order", elimination_order)
    original_feature_indices = np.arange(0, length, dtype = int)
    print("#Init feature indices", original_feature_indices)
    clf = SVC(kernel = "linear", C = C, random_state = 0)

    iter_without_dropping = 0
    n_removed_features = 0

    while iter_without_dropping < 20 and length > 10:

        # Fit the SVC and compute z

        clf.fit(final_X * np.outer(np.ones(X.shape[ 0 ]), z), Y)
        z *= np.abs(clf.coef_[0]) # In absolute value
        #clf devuelve los coeficientes w
        n_features_to_drop = np.sum(z < threshold)
        
        if n_features_to_drop == 0:
            iter_without_dropping += 1
        else:
            iter_without_dropping = 0
            remove_order = np.argsort(z[ z < threshold ])
            elimination_order[ original_feature_indices[ z < threshold ][ remove_order ] ] = \
                np.arange(0, n_features_to_drop) + n_removed_features + 1
            n_removed_features += n_features_to_drop
            length -= n_features_to_drop
        
            # Delete from X, z and original_features the selected attributes 

            final_X = final_X[ :, z >= threshold ]
            original_feature_indices = original_feature_indices[ z >= threshold ]
            z = z[ z >= threshold ]

    # We remove all remaining features

    if length > 0:
            remove_order = np.argsort(z)
            elimination_order[ original_feature_indices[ remove_order ] ] = \
                np.arange(0, length) + n_removed_features + 1

    return np.argsort(-elimination_order)  # So array starts at 0 (python indexing)


In [2]:
import sys
NROWS = sys.maxsize
#NROWS = 10

df_chandran = pd.read_csv('./data/chandran.csv', sep=',', 
                     header=0, nrows = NROWS)
df_chandran.head()


,X100_g_at,X1000_at,X1001_at,X1002_f_at,X1003_s_at,X1004_at,X1005_at,X1006_at,X1007_s_at,X1008_f_at,...,AFFX.ThrX.5_at,AFFX.ThrX.M_at,AFFX.TrpnX.3_at,AFFX.TrpnX.5_at,AFFX.TrpnX.M_at,AFFX.YEL002c.WBP1_at,AFFX.YEL018w._at,AFFX.YEL021w.URA3_at,AFFX.YEL024w.RIP1_at,Y
0,7.234793,6.494211,4.853264,3.527822,5.575283,5.630715,7.070994,3.586507,8.607721,8.376150,...,4.124928,3.130851,2.983105,3.286748,3.632831,3.200749,3.157482,3.572550,3.201209,1
1,6.967237,6.632175,4.320490,3.535030,5.505270,5.173343,7.826527,3.470474,6.871599,8.732676,...,4.089809,3.030838,2.710369,3.204168,3.721313,3.080551,2.908750,2.980353,3.264706,1
2,7.026961,6.510959,4.267634,3.387379,5.906008,5.321219,7.857653,3.292397,7.521978,8.636165,...,3.693827,2.755653,2.526112,3.254250,3.362329,2.862432,3.048200,3.247433,3.061890,1
3,7.123875,6.155900,4.114608,3.380995,5.891499,5.602339,8.285221,3.636381,8.148127,8.472201,...,4.345752,3.122182,2.656120,3.530544,3.515947,3.026449,3.231532,3.762868,3.354885,1
4,7.182206,6.237578,4.194653,3.380361,5.511587,5.383889,8.941296,3.331588,8.257033,8.700136,...,4.016990,2.956002,2.622684,3.263552,3.606437,3.035578,2.938062,3.156967,3.055146,1


In [6]:
SELECT_SAMPLES = 5
SELECT_FEATURES_INI = 100
SELECT_FEATURES_FIN = 110
labels = np.concatenate([df_chandran.iloc[0:SELECT_SAMPLES, -1], 
                         df_chandran.iloc[-SELECT_SAMPLES:, -1]], axis = 0)
#print(labels)

features = np.concatenate([df_chandran.iloc[0:SELECT_SAMPLES, SELECT_FEATURES_INI:SELECT_FEATURES_FIN], 
                           df_chandran.iloc[-SELECT_SAMPLES:, SELECT_FEATURES_INI:SELECT_FEATURES_FIN]], axis = 0)
#print(features)

TEST_SIZE = 0.33
random_state = 0
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=TEST_SIZE, random_state=random_state)

print(X_train)
print(y_train)

[[2.86445999 7.32942575 7.7506878  2.92797381 4.68776147 2.42514442
  5.85504417 4.64712179 2.69683708 4.13209918]
 [3.19159297 7.44015753 8.01133704 2.9925223  5.12199766 2.2825365
  5.78955862 5.44155007 2.69530392 3.59354046]
 [3.43724992 7.60105699 7.93612435 2.9981392  5.01502853 2.5739303
  5.82747709 4.93601557 2.86524365 4.73670692]
 [2.95738953 7.14597925 8.02878747 3.09612524 4.88298697 2.4350044
  5.69849878 4.77639802 2.7831269  5.01608014]
 [2.96428904 7.18547986 7.83712188 3.21338741 4.88506596 2.41277747
  5.65483307 4.73698059 2.75456921 3.76447161]
 [2.91762047 6.84221384 7.40046046 3.14257247 4.95956015 2.96528045
  5.80640559 4.02195149 2.80259039 2.6947983 ]]
[ 1 -1 -1  1  1 -1]


### Ranking model

In [7]:
##
# X is numpy array witht the data (rows are data instances)
# Y is a numpy vector with the class labels (-1 or 1)
# C is the regularization coefficient of the SVM
# threshold is the threshold value to drop features in L2AROM

variable_ranking(X_train, y_train, C = 1, threshold = 1e-10)

#Features 10
#Elimination order [0 0 0 0 0 0 0 0 0 0]
#Init feature indices [0 1 2 3 4 5 6 7 8 9]


array([9, 8, 7, 6, 5, 4, 3, 2, 1, 0])

In [22]:
%%bash
ls data
wc -l data/chandran.csv
#head -n 1 data/chandran.csv


chandran.csv
singh.csv
welsh.csv
     105 data/chandran.csv


# Outputs

In [14]:
%%bash
jupyter nbconvert --to=latex --template=~/report.tplx feature_selection_linear_models.ipynb 1> /dev/null
pdflatex -shell-escape feature_selection_linear_models 1> /dev/null
jupyter nbconvert --to html_with_toclenvs feature_selection_linear_models.ipynb 1> /dev/null

[NbConvertApp] Converting notebook feature_selection_linear_models.ipynb to latex
[NbConvertApp] Writing 32105 bytes to feature_selection_linear_models.tex
[NbConvertApp] Converting notebook feature_selection_linear_models.ipynb to html_with_toclenvs
[NbConvertApp] Writing 285913 bytes to feature_selection_linear_models.html
